In [ ]:
# Install depedencies
!rm -r -f * # delete all data
!pip3 install ctranslate2==2.18.0 OpenNMT-tf==2.26 pyonmttok==1.31

**Introducció**

Execució del sistema de traducció de Softcatalà https://github.com/Softcatala/nmt-models en un Jupyter notebook.


In [ ]:
# Get Softcatalà scripts (same used in real training)
!git clone --depth=1 https://github.com/Softcatala/nmt-models


In [ ]:
# Get the eng-cat corpus only (not the full corpus)
!git clone --depth=1 https://github.com/jordimas/corpus-eng-cat
!cd corpus-eng-cat && xz -f -d *.xz
!mkdir nmt-models/languages/eng-cat/corpus-raw/
!cp corpus-eng-cat/* nmt-models/languages/eng-cat/corpus-raw/
!find . -type f -name '*.ca' | xargs sed -i 's/ meva / meua /g'
!rm -r -f corpus-eng-cat/

In [ ]:
#Prepare vocabulary
%cd nmt-models/languages/eng-cat
!export QUICK_PREPROCESSING=1 && ./preprocess.sh
!./voc.sh

In [ ]:
# Train the model
!sed -i "s/80000/8000/g" data.yml
!onmt-main  --model_type TransformerTiny --config data.yml --auto_config train --with_eval

In [ ]:
#Export model in ctranslate2 format
!rm exported/ -r -f
!ct2-opennmt-tf-converter --model_path run/ --output_dir exported/ --src_vocab sp-vocab.txt.token --tgt_vocab sp-vocab.txt.token --quantization int8

In [ ]:
# Do a translation using the new model
import pyonmttok
import ctranslate2

source = "My sister is older."

tokenizer=pyonmttok.Tokenizer(mode="none", sp_model_path = "sp_m.model")
tokenized=tokenizer.tokenize(source)

translator = ctranslate2.Translator("exported/")
translated = translator.translate_batch([tokenized[0]])
translated = tokenizer.detokenize(translated[0][0]['tokens'])
print(f"Source: {source}")
print(f"Translation: {translated}")